# Testbed initializaton

Read configuration

In [ ]:
from testbed_showcase.constants import EnumFut, EnumTentacleType
from testbed_showcase.util_testbed import get_testbed

fut = EnumFut.FUT_I2C

TESTBED = get_testbed()

Select 3 tentacles - As there are two MCU tentacles connected, we specify one by its serial

In [ ]:
mcu = TESTBED.get_tentacle(
    tentacle_type=EnumTentacleType.TENTACLE_MCU,
    # serial="e46340474b4e1831",   # pico
    serial="e46340474b551722",  # pyboard
)
daq = TESTBED.get_tentacle(tentacle_type=EnumTentacleType.TENTACLE_DAQ_SALEAE)
potpourry = TESTBED.get_tentacle(tentacle_type=EnumTentacleType.TENTACLE_DEVICE_POTPOURRY)

tentacles = [mcu, daq, potpourry]

# Tentacle Infra

Connect and powercycle

In [ ]:
from octoprobe.octoprobe import CtxTestRun

# INFRA: Powercycle all tentacles
CtxTestRun.session_powercycle_tentacles()

# INFRA: Initialize tentacles
[t.infra.setup_infra(udev=None) for t in tentacles]

Communicate with the tentacle infra MCU

In [ ]:
mcu.infra.mcu_infra.get_micropython_version()

## Tentacle Relays

Close tentalces relays to allow I2C testing

In [ ]:
fut = EnumFut.FUT_I2C
for t in tentacles:
    t.set_relays_by_FUT(fut=fut, open_others=True)

Explicitely open/close the relays

In [ ]:
mcu.infra.mcu_infra.relays(relays_close=[2, 7], relays_open=[1,2,3,4,5,6,7])

# Tentacle DUT

Flash the MCU soldered on tentacles DUT

**RPI_PICO2 / RPI_PICO / PYBV11**

In [ ]:
import pathlib

from octoprobe.util_firmware_spec import FirmwareDownloadSpec
from octoprobe.util_pyudev import UdevPoller

PYBV11 = {
    "board_variant": "PYBV11",
    "url": "https://micropython.org/resources/firmware/PYBV11-20240602-v1.23.0.dfu",
    "micropython_version_text": "3.4.0; MicroPython v1.23.0 on 2024-06-02"
}
RPI_PICO2 = {
    "board_variant": "RPI_PICO2",
    "url": "https://micropython.org/resources/firmware/RPI_PICO2-20241025-v1.24.0.uf2",
    "micropython_full_version_text": "3.4.0; MicroPython v1.24.0 on 2024-10-25;Raspberry Pi Pico2 with RP2350"
}
RPI_PICO = {
    "board_variant": "RPI_PICO",
    "url": "https://micropython.org/resources/firmware/RPI_PICO-20240602-v1.23.0.uf2",
    "micropython_full_version_text": "3.4.0; MicroPython v1.23.0 on 2024-06-02;Raspberry Pi Pico with RP2040"
}
firmware_spec=FirmwareDownloadSpec.factory_json(RPI_PICO)
# TODO: Raise exception if board_variant does not equal
mcu.power.dut = True # TODO: This line should not be required
mcu.dut.mp_remote_close() # TODO: This line should not be required
with UdevPoller() as udev_poller:
    mcu.flash_dut(udev_poller=udev_poller, firmware_spec=firmware_spec, directory_logs=pathlib.Path("/tmp"))

Power on DUT MCU and wait for USB device to appear

In [ ]:
mcu.dut.mp_remote_close() # TODO: This line should not be required
mcu.power.dut = True # TODO: This line should not be required
with UdevPoller() as udev_poller:
    mcu.dut_boot_and_init_mp_remote(udev=udev_poller)

Free serial port and return port name. Now we may connect using Thonny

In [ ]:
mcu.dut.mp_remote_close()

DUT MCU: Run some micropython code: blink red led

In [ ]:
PYBV11 = """
from pyb import Timer

led =  pyb.LED(1)
tim = Timer(1, freq=1000)
tim.freq(2.0)
tim.callback(lambda _: led.toggle())
"""

RPI_PICO = """
from machine import Timer, Pin

led = Pin(25, Pin.OUT)
tim = Timer(period=20, callback=lambda _: led.toggle())
"""

mcu.dut.mp_remote.exec_raw(RPI_PICO)
